In [1]:
!pip install sentence_transformers datasets accelerate

# base

In [2]:
!pip install -U datasets

In [3]:
import random
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from datasets import load_dataset

In [26]:
corpus = load_dataset("khanglt0004/vietnamese_legal_chunks", split = "train")
queries = load_dataset("khanglt0004/questions", split = "train")
relevant_docs_data = load_dataset("khanglt0004/links", split = "train")
# Convert the datasets to dictionaries
corpuss = dict(zip(corpus["id"], corpus["text"]))  # Our corpus (cid => document)
corpus = {}
for cid, text in corpuss.items():
    cid = str(cid)
    corpus[cid] = text
queries = dict(zip(queries["qid"], queries["question"]))  # Our queries (qid => question)
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for qid, corpus_ids in zip(relevant_docs_data["q_id"], relevant_docs_data["chunk_id"]):
    qid = str(qid)
    corpus_ids = str(corpus_ids)
    if qid not in relevant_docs:
        relevant_docs[qid] = set()
    relevant_docs[qid].add(corpus_ids)

In [28]:
import json
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets

model = SentenceTransformer("bkai-foundation-models/vietnamese-bi-encoder")
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

In [29]:
# Evaluate the model
results = evaluator(model)
for k,v in results.items():
    print(k, v)

dim_768_cosine_accuracy@1 0.4070760471736478
dim_768_cosine_accuracy@3 0.607157381049207
dim_768_cosine_accuracy@5 0.6856445709638064
dim_768_cosine_accuracy@10 0.7669784465229769
dim_768_cosine_precision@1 0.4070760471736478
dim_768_cosine_precision@3 0.20238579368306897
dim_768_cosine_precision@5 0.13712891419276127
dim_768_cosine_precision@10 0.07669784465229768
dim_768_cosine_recall@1 0.4070760471736478
dim_768_cosine_recall@3 0.607157381049207
dim_768_cosine_recall@5 0.6856445709638064
dim_768_cosine_recall@10 0.7669784465229769
dim_768_cosine_ndcg@10 0.5845575816547848
dim_768_cosine_mrr@10 0.5264370598449489
dim_768_cosine_map@100 0.5332781076975427
dim_512_cosine_accuracy@1 0.4062627084180561
dim_512_cosine_accuracy@3 0.601870679137861
dim_512_cosine_accuracy@5 0.6738511590077267
dim_512_cosine_accuracy@10 0.7559983733224889
dim_512_cosine_precision@1 0.4062627084180561
dim_512_cosine_precision@3 0.2006235597126203
dim_512_cosine_precision@5 0.13477023180154535
dim_512_cosine_p

# Training

In [30]:
import pandas as pd
from datasets import Dataset

def prepare_training_dataset(queries, corpus, relevant_docs):
    anchors = []
    positives = []
    for query_id, docs in relevant_docs.items():
        for doc_id in docs:
          anchors.append(queries[query_id])
          positives.append(corpus[str(doc_id)] )
    df = {
        "anchor": anchors,
        "positive": positives
    }

    return Dataset.from_dict(df)

pairs = prepare_training_dataset(queries, corpus, relevant_docs)
pairs

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 2459
})

In [31]:
pairs[0]

{'anchor': 'Quy định này áp dụng cho những đối tượng nào liên quan đến chính sách dân số và kế hoạch hóa gia đình?',
 'positive': 'Đối tượng áp dụng\n\nQuy định này quy định tiêu chuẩn, điều kiện, thẩm quyền xem xét kết nạp lại vào Đảng đối với đảng viên đã bị đưa ra khỏi Đảng do vi phạm chính sách dân số và kế hoạch hoá gia đình, kết nạp quần chúng vi phạm chính sách dân số và kế hoạch hoá gia đình có nguyện vọng phấn đấu vào Đảng.\n\nĐiều 2. Những trường hợp sinh con không bị coi là vi phạm chính sách dân số và kế hoạch hoá gia đình\n\n1. Cặp vợ chồng sinh con thứ ba, nếu cả hai hoặc một trong hai người thuộc dân tộc có số dân dưới 10.000 người hoặc thuộc dân tộc có nguy cơ suy giảm số dân (tỉ lệ nhỏ hơn hoặc bằng tỉ lệ chết) theo công bố chính thức của Bộ Kế hoạch và Đầu tư.\n\n2. Cặp vợ chồng sinh lần thứ nhất mà sinh ba con trở lên.\n\n3. Cặp vợ chồng đã có một con đẻ, sinh lần thứ hai mà sinh hai con trở lên.\n\n4. Cặp vợ chồng sinh lần thứ ba trở lên, nếu tại thời điểm sinh chỉ 

In [32]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [33]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="sample", # output directory and hugging face model ID
    num_train_epochs=1,                         # number of epochs
    per_device_train_batch_size=8,             # train batch size
    gradient_accumulation_steps=4,             # for a global batch size of 512
    per_device_eval_batch_size=4,              # evaluation batch size
    #gradient_checkpointing=True,
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    #tf32=True,                                  # use tf32 precision
    bf16=True,                                  # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="steps",                      # evaluate after each epoch
    #save_strategy="epoch",                      # save after each epoch
    save_steps = 500,
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    metric_for_best_model="eval_dim_768_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
)

In [34]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,  # training arguments
    train_dataset=pairs,
    loss=train_loss,
    evaluator=evaluator,
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [35]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save the best model
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: luu92241 (luu92241-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
10,5.205800,No log,0.451810,0.665311,0.736885,0.812119,0.451810,0.221770,0.147377,0.081212,0.451810,0.665311,0.736885,0.812119,0.631425,0.573517,0.579415,0.446116,0.653924,0.727125,0.803985,0.446116,0.217975,0.145425,0.080399,0.446116,0.653924,0.727125,0.803985,0.624372,0.566920,0.573096,0.414396,0.636844,0.707605,0.786905,0.414396,0.212281,0.141521,0.078691,0.414396,0.636844,0.707605,0.786905,0.600391,0.540635,0.546959,0.390809,0.588044,0.672631,0.760065,0.390809,0.196015,0.134526,0.076007,0.390809,0.588044,0.672631,0.760065,0.571187,0.511191,0.517903,0.321269,0.501017,0.572997,0.666124,0.321269,0.167006,0.114599,0.066612,0.321269,0.501017,0.572997,0.666124,0.487354,0.430866,0.439904,0.487354
20,3.397600,No log,0.472956,0.688898,0.759658,0.840179,0.472956,0.229633,0.151932,0.084018,0.472956,0.688898,0.759658,0.840179,0.655933,0.596966,0.602303,0.476210,0.681578,0.753152,0.834892,0.476210,0.227193,0.150630,0.083489,0.476210,0.681578,0.753152,0.834892,0.653377,0.595467,0.600970,0.443676,0.661244,0.739325,0.816999,0.443676,0.220415,0.147865,0.081700,0.443676,0.661244,0.739325,0.816999,0.629497,0.569503,0.575349,0.416836,0.630744,0.700691,0.786499,0.416836,0.210248,0.140138,0.078650,0.416836,0.630744,0.700691,0.786499,0.598614,0.538793,0.545439,0.352582,0.551037,0.615291,0.707605,0.352582,0.183679,0.123058,0.070760,0.352582,0.551037,0.615291,0.707605,0.525111,0.467267,0.476059,0.525111
30,3.429100,No log,0.470923,0.694591,0.771858,0.843839,0.470923,0.231530,0.154372,0.084384,0.470923,0.694591,0.771858,0.843839,0.657437,0.597594,0.603228,0.477837,0.687271,0.764132,0.840992,0.477837,0.229090,0.152826,0.084099,0.477837,0.687271,0.764132,0.840992,0.657704,0.599101,0.604738,0.445303,0.672224,0.747458,0.825539,0.445303,0.224075,0.149492,0.082554,0.445303,0.672224,0.747458,0.825539,0.635492,0.574523,0.580499,0.422123,0.640911,0.716145,0.799512,0.422123,0.213637,0.143229,0.079951,0.422123,0.640911,0.716145,0.799512,0.608445,0.547493,0.554166,0.368849,0.563644,0.637658,0.729972,0.368849,0.187881,0.127532,0.072997,0.368849,0.563644,0.637658,0.72

# Re-evaluate

In [36]:
from sentence_transformers import SentenceTransformer
import torch
fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluate the model
results = evaluator(fine_tuned_model)

for k,v in results.items():
    print(k, v)

dim_768_cosine_accuracy@1 0.48393655957706383
dim_768_cosine_accuracy@3 0.7157381049206994
dim_768_cosine_accuracy@5 0.7897519316795445
dim_768_cosine_accuracy@10 0.8605124034160228
dim_768_cosine_precision@1 0.48393655957706383
dim_768_cosine_precision@3 0.23857936830689977
dim_768_cosine_precision@5 0.1579503863359089
dim_768_cosine_precision@10 0.08605124034160226
dim_768_cosine_recall@1 0.48393655957706383
dim_768_cosine_recall@3 0.7157381049206994
dim_768_cosine_recall@5 0.7897519316795445
dim_768_cosine_recall@10 0.8605124034160228
dim_768_cosine_ndcg@10 0.674188755112397
dim_768_cosine_mrr@10 0.6141569356494113
dim_768_cosine_map@100 0.6194318226528878
dim_512_cosine_accuracy@1 0.4843432289548597
dim_512_cosine_accuracy@3 0.7092313948759659
dim_512_cosine_accuracy@5 0.7864985766571777
dim_512_cosine_accuracy@10 0.862545750305002
dim_512_cosine_precision@1 0.4843432289548597
dim_512_cosine_precision@3 0.23641046495865525
dim_512_cosine_precision@5 0.15729971533143552
dim_512_cosi

# Push model to hub

In [ ]:
from huggingface_hub import login

login(token="", add_to_git_credential=True)  # ADD YOUR TOKEN HERE

In [38]:
# push model to hub
trainer.model.push_to_hub("ltk_embedding")

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

'https://huggingface.co/khanglt0004/ltk_embedding/commit/f4d64f308e223af63fcb5cf3fe3f1ca4ef6f6d0b'